# Data Cleaning Notebook
**Objective:** Load, inspect, clean, and export `online_retail.xlsx`  
**Issues to document:** nulls, duplicates, negatives, outlier

In [1]:
import pandas as pd
import numpy as np

In [4]:
# Use forward slashes for the Windows path (works in Python) to avoid escape issues
raw_path = 'E:/FARIS WORK/data-analyst-intern-assignment/raw_data/Online Retail.xlsx'
df = pd.read_excel(raw_path)


In [5]:
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nSample:")
print(df.head(3))
print("\nInfo:")
df.info()

Shape: (541909, 8)

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']

Sample:
  InvoiceNo StockCode                         Description  Quantity  \
0    536365    85123A  WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                 WHITE METAL LANTERN         6   
2    536365    84406B      CREAM CUPID HEARTS COAT HANGER         8   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 

## Data Quality Issues Found

| Issue | Description | Count | % of Total |
|-------|-----------|--------|------------|
| **Missing CustomerID** | `CustomerID` is null → cannot link to customer | `df['CustomerID'].isna().sum()` | ~25% |
| **Cancellations** | `InvoiceNo` starts with 'C' OR `Quantity < 0` | `df[df['InvoiceNo'].astype(str).str.startswith('C')].shape[0]` | ~16k–20k rows |
| **Negative UnitPrice** | Price ≤ 0 (invalid for sales) | `df[df['UnitPrice'] <= 0].shape[0]` | ~2k–3k |
| **Duplicates** | Exact duplicate rows | `df.duplicated().sum()` | ~5k |
| **Outliers** | Extreme `Quantity` or `UnitPrice` (e.g., >10,000 units or >£5,000) | Use quantiles | TBD |
| **Data Types** | `InvoiceDate` is object → needs datetime <br> `CustomerID` is float → should be int/string | `df.dtypes` | N/A |

In [7]:
# QUICK ISSUE CHECKS
print("Missing CustomerID:", df['CustomerID'].isna().sum(), f"({df['CustomerID'].isna().mean():.1%})")


Missing CustomerID: 135080 (24.9%)


In [8]:
print("Cancellations (C prefix):", df[df['InvoiceNo'].astype(str).str.startswith('C')].shape[0])


Cancellations (C prefix): 9288


In [9]:
print("Negative Quantity:", (df['Quantity'] < 0).sum())


Negative Quantity: 10624


In [10]:
print("Zero/Negative UnitPrice:", (df['UnitPrice'] <= 0).sum())


Zero/Negative UnitPrice: 2517


In [11]:
print("Duplicates:", df.duplicated().sum())


Duplicates: 5268


In [14]:
print("\nOutlier candidates:")
print("  Quantity > 10,000:", (df['Quantity'] > 10000).sum())
print("  UnitPrice > £5,000:", (df['UnitPrice'] > 5000).sum())


Outlier candidates:
  Quantity > 10,000: 3
  UnitPrice > £5,000: 31


## Transformation 1: Data Types + Revenue Column
- Convert `InvoiceDate` → datetime  
- `CustomerID` → string (for safety)  
- Create `Revenue = Quantity * UnitPrice`  

In [15]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], dayfirst=True)
df['CustomerID'] = df['CustomerID'].astype('str')

In [16]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

In [17]:
print("InvoiceDate dtype:", df['InvoiceDate'].dtype)
print("Sample Revenue:", df['Revenue'].head(3).tolist())

InvoiceDate dtype: datetime64[ns]
Sample Revenue: [15.299999999999999, 20.34, 22.0]


## Transformation 2: Remove Duplicates
- **Issue:** 5,268 exact duplicate rows  
- **Fix:** Drop duplicates (keep first)  
- **Validation:** Confirm count = 0

In [18]:
print("Rows before dedupe:", len(df))

Rows before dedupe: 541909


In [19]:
df = df.drop_duplicates(keep='first')

In [20]:
print("Rows after dedupe:", len(df))
print("Remaining duplicates:", df.duplicated().sum())
assert df.duplicated().sum() == 0, "Duplicates exist!"

Rows after dedupe: 536641
Remaining duplicates: 0
Remaining duplicates: 0


## Transformation 3: Cancellations & Invalid Prices
- **Cancellations:** `InvoiceNo` starts with 'C' AND `Quantity < 0` → keep for revenue calc (negative revenue)  
- **Invalid:** `UnitPrice <= 0` OR `Quantity <= 0` without 'C' → remove (likely errors)  
- **Keep cancellations** for lost revenue insight  
- **Validation:** Count remaining invalid rows = 0

In [21]:
print("Rows before invalid removal:", len(df))
print("Cancellations:", df[df['InvoiceNo'].astype(str).str.startswith('C')].shape[0])

Rows before invalid removal: 536641
Cancellations: 9251
Cancellations: 9251


In [22]:
df['IsCancellation'] = df['InvoiceNo'].astype(str).str.startswith('C')

In [23]:
invalid = df[
    (~df['IsCancellation']) & 
    ((df['UnitPrice'] <= 0) | (df['Quantity'] <= 0))
]
print("Invalid rows to remove:", len(invalid))

df = df.drop(invalid.index)

Invalid rows to remove: 2512


In [24]:
remaining_invalid = df[
    (~df['IsCancellation']) & 
    ((df['UnitPrice'] <= 0) | (df['Quantity'] <= 0))
]
print("Remaining invalid:", len(remaining_invalid))
assert len(remaining_invalid) == 0, "Invalid rows remain!"

Remaining invalid: 0


In [25]:
print("Final rows after invalid removal:", len(df))

Final rows after invalid removal: 534129


## Transformation 4: Handle Missing CustomerID
- **Issue:** ~135k rows (~25%) have null `CustomerID` → cannot do customer-level analysis  
- **Fix:**  
  - **Option 1 (Chosen):** Keep all rows for **aggregate insights** (revenue, product, country)  
  - Create two versions:  
    - `df_full`: All cleaned data  
    - `df_customer`: Only rows with valid `CustomerID` (for CLV, RFM)  
- **Validation:** Confirm no nulls in `df_customer['CustomerID']`

In [26]:
df_full = df.copy()

In [27]:
df_customer = df.dropna(subset=['CustomerID']).copy()
df_customer['CustomerID'] = df_customer['CustomerID'].astype('str').str.replace('.0', '', regex=False)

In [28]:
print("df_full rows:", len(df_full))
print("df_customer rows:", len(df_customer))
print("Null CustomerID in df_customer:", df_customer['CustomerID'].isna().sum())
assert df_customer['CustomerID'].notna().all(), "Null CustomerID remains!"

print("Sample CustomerID:", df_customer['CustomerID'].head(3).tolist())

df_full rows: 534129
df_customer rows: 534129
Null CustomerID in df_customer: 0
Sample CustomerID: ['17850', '17850', '17850']


## Transformation 5: Remove Outliers
- **Issue:** Extreme values distort revenue (e.g., Quantity > 10,000 or UnitPrice > £5,000)  
- **Fix:** Cap at 99th percentile (conservative) OR remove > 99.5th  
- **Chosen:** Remove rows where `Quantity > 99.5th` OR `UnitPrice > 99.5th`  
- **Validation:** Confirm no values above threshold

In [29]:
qty_995 = df_full['Quantity'].quantile(0.995)
price_995 = df_full['UnitPrice'].quantile(0.995)
print(f"99.5th percentile → Quantity: {qty_995:.0f}, UnitPrice: £{price_995:.2f}")

99.5th percentile → Quantity: 160, UnitPrice: £24.96


In [30]:
print("Rows before outlier removal:", len(df_full))

Rows before outlier removal: 534129


In [31]:
df_full = df_full[
    (df_full['Quantity'] <= qty_995) &
    (df_full['UnitPrice'] <= price_995)
].copy()

In [32]:
print("Rows after (df_full):", len(df_full))
assert (df_full['Quantity'] <= qty_995).all(), "Quantity outliers remain!"
assert (df_full['UnitPrice'] <= price_995).all(), "Price outliers remain!"

Rows after (df_full): 528887


## Transformation 6: Final Validation & Export
- **Summary of cleaning steps applied**  
- **Final checks:** No nulls in key fields, positive revenue in non-cancellations, date range  
- **Export:**  
  - `df_full` → `../cleaned_data/retail_clean_full.csv`  
  - `df_customer` → `../cleaned_data/retail_clean_customer.csv`  
- **Validation:** File exists + row count matches

In [33]:
print("=== FINAL DATA SUMMARY ===")
print(f"df_full: {len(df_full):,} rows")
print(f"df_customer: {len(df_customer):,} rows")
print(f"Date range: {df_full['InvoiceDate'].min()} to {df_full['InvoiceDate'].max()}")
print(f"Total Revenue (incl. cancellations): £{df_full['Revenue'].sum():,.2f}")

=== FINAL DATA SUMMARY ===
df_full: 528,887 rows
df_customer: 534,129 rows
Date range: 2010-12-01 08:26:00 to 2011-12-09 12:50:00
Total Revenue (incl. cancellations): £8,185,493.07


In [34]:
assert (df_full.loc[~df_full['IsCancellation'], 'Quantity'] > 0).all(), "Non-cancellation has <=0 qty!"
assert (df_full.loc[~df_full['IsCancellation'], 'UnitPrice'] > 0).all(), "Non-cancellation has <=0 price!"
assert df_full['InvoiceDate'].notna().all(), "Missing dates!"

In [35]:
full_path = 'E:/FARIS WORK/data-analyst-intern-assignment/Cleaned_data/retail_clean_full.csv'
customer_path = 'E:/FARIS WORK/data-analyst-intern-assignment/Cleaned_data/retail_clean_customer.csv'

In [36]:
df_full.to_csv(full_path, index=False)
df_customer.to_csv(customer_path, index=False)

In [37]:
print(f"\nExported: {full_path}")
print(f"Exported: {customer_path}")


Exported: E:/FARIS WORK/data-analyst-intern-assignment/Cleaned_data/retail_clean_full.csv
Exported: E:/FARIS WORK/data-analyst-intern-assignment/Cleaned_data/retail_clean_customer.csv


In [38]:
import os
assert os.path.exists(full_path), "Full CSV not saved!"
assert os.path.exists(customer_path), "Customer CSV not saved!"
print("All validation passed. Cleaning COMPLETE.")

All validation passed. Cleaning COMPLETE.
